# Pytorch 시각화를 위한 tensorboardX 사용법

190509, keep-steady

https://github.com/lanpa/tensorboardX

In [1]:
# 설치
# tensorflow가 설치되어 있어야 한다
!pip install tensorboardX

You are using pip version 18.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# 서버 사용시 터미널창에 아래 명령어 입력
> tensorboard --logdir runs

In [4]:
# demo.py

import torch
import torchvision.utils as vutils
import numpy as np
import torchvision.models as models
from torchvision import datasets
from tensorboardX import SummaryWriter

resnet18 = models.resnet18(False)
writer = SummaryWriter()
sample_rate = 44100
freqs = [262, 294, 330, 349, 392, 440, 440, 440, 440, 440, 440]

for n_iter in range(100):

    dummy_s1 = torch.rand(1)
    dummy_s2 = torch.rand(1)
    # data grouping by `slash`
    writer.add_scalar('loss/L1_loss', dummy_s1[0], n_iter)
    
    writer.add_scalars('loss/L1+L2_loss', {'xsinx': n_iter * np.sin(n_iter),
                                             'xcosx': n_iter * np.cos(n_iter)}, n_iter)

    dummy_img = torch.rand(32, 3, 64, 64)  # output from network
    if n_iter % 10 == 0:
        x = vutils.make_grid(dummy_img, normalize=True, scale_each=True)
        writer.add_image('Image', x, n_iter)

        dummy_audio = torch.zeros(sample_rate * 2)
        for i in range(x.size(0)):
            # amplitude of sound should in [-1, 1]
            dummy_audio[i] = np.cos(freqs[n_iter // 10] * np.pi * float(i) / float(sample_rate))
        writer.add_audio('myAudio', dummy_audio, n_iter, sample_rate=sample_rate)

        writer.add_text('Text', 'text logged at step:' + str(n_iter), n_iter)

        for name, param in resnet18.named_parameters():
            writer.add_histogram(name, param.clone().cpu().data.numpy(), n_iter)

        # needs tensorboard 0.4RC or later
        writer.add_pr_curve('xoxo', np.random.randint(2, size=100), np.random.rand(100), n_iter)

dataset = datasets.MNIST('mnist', train=False, download=False)
images  = dataset.test_data[:100].float()
label   = dataset.test_labels[:100]

features = images.view(100, 784)
writer.add_embedding(features, metadata=label, label_img=images.unsqueeze(1))

# export scalar data to JSON for external processing
writer.export_scalars_to_json("./all_scalars.json")
writer.close()

In [26]:
writer = SummaryWriter()

x = torch.rand(3, 64, 64)  # output from network
writer.add_image('testt/torch_image', x, n_iter)

x = np.zeros((3, 64, 64))  # output from network
writer.add_image('testt/numpy_image', x, n_iter)

x = torch.rand(10, 3, 64, 64)  # output from network
x = vutils.make_grid(x, normalize=True, scale_each=True)
writer.add_image('testt/bulk_image', x, n_iter)

In [3]:
from numpy.random import rand
from tensorboardX import SummaryWriter
import time


with SummaryWriter() as writer:
    for n_iter in range(100):
        writer.add_scalar('twse/0050', rand(), n_iter)
        writer.add_scalar('twse/2330', rand(), n_iter)
        t = rand()
        writer.add_scalar('dow/aaa', t, n_iter)
        writer.add_scalar('dow/bbb', t - 1, n_iter)
        writer.add_scalar('dow/ccc', t + 1, n_iter)
        writer.add_scalar('nasdaq/aaa', rand(), n_iter)
        writer.add_scalar('nasdaq/bbb', rand(), n_iter)
        writer.add_scalar('nasdaq/ccc', rand(), n_iter)

    layout = {'Taiwan': {'twse': ['Multiline', ['twse/0050', 'twse/2330']]},
              'USA': {'dow': ['Margin', ['dow/aaa', 'dow/bbb', 'dow/ccc']],
                      'nasdaq': ['Margin', ['nasdaq/aaa', 'nasdaq/bbb', 'nasdaq/ccc']]}}
    writer.add_custom_scalars(layout)
#    writer.add_custom_scalars(layout) second call has no effect

time.sleep(1)

with SummaryWriter() as writer:
    for n_iter in range(100):
        writer.add_scalar('twse/0050', rand(), n_iter)
        writer.add_scalar('twse/2330', rand(), n_iter)

    writer.add_custom_scalars_multilinechart(['twse/0050', 'twse/2330'])

time.sleep(1)

with SummaryWriter() as writer:
    for n_iter in range(100):
        t = rand()
        writer.add_scalar('dow/aaa', t, n_iter)
        writer.add_scalar('dow/bbb', t - 1, n_iter)
        writer.add_scalar('dow/ccc', t + 1, n_iter)

    writer.add_custom_scalars_marginchart(['dow/aaa', 'dow/bbb', 'dow/ccc'])